# Advent of Code 2021 - Day 21

In [17]:
interface Dice {
    val rolls: Int
    fun roll(): Int
}

class DeterministicDice : Dice {
    override var rolls: Int = 0
        private set

    override fun roll(): Int = ((rolls % 100) + 1).also { rolls++ }
}

data class Player(val position: Int, val score: Int = 0)

val BOARD: IntRange = 1..10

In [18]:
import java.io.File

val (player1: Player, player2: Player) = File("Day21.input.txt")
    .bufferedReader()
    .readLines()
    .map { Regex(""".+: (\d+)""").find(it)!!.destructured }
    .map { (pos) -> pos.toInt() }
    .map { pos -> Player(position = pos) }

## Part 1

Play a game between `player1` and `player2` with a `DeterministicDice`. Players alternate turns and the game ends immediately when either player's score reaches 1000. During a turn the player will `Dice:roll` three times. The result moves their current `Player::position` along the `BOARD` and the position they land on is incremented to their `Player::score`. What is the product of the number of `Dice::rolls` and the losing `Player::score`.

In [19]:
fun Player.move(dice: Dice): Player {
    val pos = ((position + dice.roll() + dice.roll() + dice.roll() - 1) % BOARD.endInclusive) + 1
    return Player(
        position = pos,
        score = score + pos
    )
}

fun Pair<Player, Player>.playGame(dice: Dice, score: Int): Pair<Player, Player> =
    first.move(dice).let { if (it.score >= score) it to second else (second to it).playGame(dice, score) }

fun Pair<Player, Player>.calculateScore(dice: Dice): Int = min(first.score, second.score) * dice.rolls

DeterministicDice().let {
    (player1 to player2)
        .playGame(it, 1000)
        .calculateScore(it)
}

684495

### Notes

This one utilizes a lot of modulo to do the wrapping. To "start" at 1 rather than 0, we can use the formula `((n - 1) % MAX) + 1`. We can then break down the game between the `Player::move` and `Pair<Player, Player>::playGame` functions.

## Part 2

Play the game with a `DiracDice`, which always returns 1, 2, and 3. The game branches and continues at every possibility on every `DiracDice::roll`. The game now ends when either `Player::score` reaches 21. Calculate the player who wins the most in every game branch. How many times did they win?

In [20]:
object DiracDice {
    fun roll(): List<Int> = listOf(1,2,3)
}

fun Player.diracMove(): List<Player> = 
    DiracDice.roll().flatMap { first ->
        DiracDice.roll().flatMap { second ->
            DiracDice.roll().map { third ->
                ((position + first + second + third - 1) % BOARD.endInclusive) + 1
            }
        }
    }.let {
        it.map {
            Player(
                position = it,
                score = score + it
            )
        }
    }

data class Wins(val player1: Long, val player2: Long)
operator fun Wins.plus(other: Wins): Wins = Wins(player1 + other.player1, player2 + other.player2)

enum class Turn { First, Second }

class Memoize(private val fn: (Player, Player, Turn) -> Wins): (Player, Player, Turn) -> Wins {
    private val memos: MutableMap<Pair<Pair<Player, Player>, Turn>, Wins> = mutableMapOf()
    override operator fun invoke(p1: Player, p2: Player, turn: Turn): Wins = 
        memos.getOrPut(((p1 to p2) to turn)) { fn(p1, p2, turn) }
}

val memoizedDiracGame = Memoize(::diracGame)

fun diracGame(player1: Player, player2: Player, turn: Turn): Wins {
    if (player1.score >= 21) return Wins(1,0)
    if (player2.score >= 21) return Wins(0,1)

    return when (turn) {
        Turn.First -> player1.diracMove().fold(Wins(0,0)) { acc, p1 ->
            acc + memoizedDiracGame(p1, player2, Turn.Second)
        }
        Turn.Second -> player2.diracMove().fold(Wins(0,0)) { acc, p2 ->
            acc + memoizedDiracGame(player1, p2, Turn.First)
        }
    }
}

memoizedDiracGame(player1, player2, Turn.First)

Wins(player1=138289532619163, player2=152587196649184)

### Notes

Didn't predict the part 2 at all, so the only thing useful from part 1 is the `Player` class. However, we could refactor part 1 to be solvable by part 2 by changing the contract of `Dice` to return a `List<Int>` on a `Dice::roll`. A `DeterministicDice` would just return a single element.

To model the "branches", rather than returning a single `Player` state (the position and score after a move) we can return a `List<Player>` states to use as the point to continue the game from. To calculate the states is just the permutations of the sum of all 3 dice rolls returning 1, 2, and 3 each.

To play the game efficiently, we need to utilize memoization. We can cache the result of how many wins a certain "game state" achieves so that if and when we encounter it again, we can quickly aggregate the wins rather than recalculating it. A game state can be quickly "serialized" into a key by identifying that the variables that affect the game's outcome are both players position and current score and whose turn it is.